# Importaciones

In [ ]:
import pandas as pd
import yfinance as yf
from alpha_vantage.timeseries import TimeSeries
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt


In [ ]:
load_dotenv()  # carga el archivo .env en el entorno
api_key = os.getenv("ALPHAVANTAGE_API_KEY")

if not api_key:
    raise RuntimeError("No se encontró la clave ALPHAVANTAGE_API_KEY. Revisa tu archivo .env.")

print("API key cargada correctamente ✅")


In [ ]:
# Datos desde Yahoo Finance
df_yahoo = yf.download("AAPL", start="2023-01-01", end="2024-12-31")

# Normalizamos las columnas y el índice
df_yahoo = df_yahoo.rename(columns=str.lower).sort_index()

print("Yahoo Finance:")
display(df_yahoo.head())
print("Fechas:", df_yahoo.index.min(), "→", df_yahoo.index.max())


In [ ]:
ts = TimeSeries(key=api_key, output_format="pandas")

# Datos diarios (outputsize='compact' → ~100 días, 'full' → histórico completo)
df_alpha, meta = ts.get_daily(symbol="AAPL", outputsize="full")


In [ ]:
df_alpha

In [ ]:
# Datos diarios (outputsize='compact' → ~100 días, 'full' → histórico completo)
df_alpha_2, meta = ts.get_daily_adjusted(symbol="AAPL", outputsize="compact")

In [ ]:
df_alpha_2, meta = ts.get_weekly_adjusted(symbol="AAPL")

In [ ]:
# Normalizamos formato
df_alpha = df_alpha.rename(
    columns={
        "1. open": "open",
        "2. high": "high",
        "3. low": "low",
        "4. close": "close",
        "5. volume": "volume"
    }
).sort_index()

print("Alpha Vantage:")
display(df_alpha.head())
print("Fechas:", df_alpha.index.min(), "→", df_alpha.index.max())

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df_yahoo["close"], label="Yahoo Finance")
plt.plot(df_alpha["close"], label="Alpha Vantage")
plt.title("Comparativa de precios de cierre - AAPL")
plt.xlabel("Fecha")
plt.ylabel("Precio cierre ($)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
summary = pd.DataFrame({
    "Fuente": ["Yahoo Finance", "Alpha Vantage"],
    "Columnas": [list(df_yahoo.columns), list(df_alpha.columns)],
    "Rango de fechas": [
        f"{df_yahoo.index.min().date()} → {df_yahoo.index.max().date()}",
        f"{df_alpha.index.min().date()} → {df_alpha.index.max().date()}",
    ],
    "Número de filas": [len(df_yahoo), len(df_alpha)]
})

display(summary)


In [ ]:
def estandarizar(df):
    return df[["open", "high", "low", "close", "volume"]].copy().sort_index()

df_yahoo_std = estandarizar(df_yahoo)
df_alpha_std = estandarizar(df_alpha)

print(df_yahoo_std.head(2))
print(df_alpha_std.head(2))


In [ ]:
df_yahoo.columns

In [ ]:
from src.data.fundamentales_yahoo import YahooFundamentals

# Crear instancia
fundamentals = YahooFundamentals("AAPL")

# Obtener balance sheet anual
balance = fundamentals.get_balance_sheet()
print(balance.head())

# Obtener balance sheet trimestral
balance_q = fundamentals.get_balance_sheet(quarterly=True)

# Ver activos totales a través del tiempo
print(balance.loc["Total Assets"])

# Obtener otros estados financieros
income = fundamentals.get_income_statement()
cash_flow = fundamentals.get_cash_flow()

# Ver resumen de la empresa
fundamentals.resumen()